#NLP GROUP 1 PROJECT
Farah Pankhania, Anushka Rohra, Angel Urbano, Weifei Wang, Geetanjali Sachdev

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = '/content/drive/MyDrive/NLP/Group_Project'
os.chdir(BASE_DIR)

# **CASE I – Business Intelligence in Social Customer Service**

In [ ]:
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

# common imports
import pandas as pd
import numpy as np
import math
import re
import glob
import os
import sys
import json
import random
import pprint as pp
import textwrap
import sqlite3
import logging

import spacy
import nltk

from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()

# pandas display options
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#available-options
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
pd.options.display.max_colwidth = 200 # default 50; -1 = all
# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# np.set_printoptions(edgeitems=3) # default 3

import matplotlib
from matplotlib import pyplot as plt

plot_params = {'figure.figsize': (8, 4), 
               'axes.labelsize': 'large',
               'axes.titlesize': 'large',
               'xtick.labelsize': 'large',
               'ytick.labelsize':'large',
               'figure.dpi': 100}
# adjust matplotlib defaults
matplotlib.rcParams.update(plot_params)

import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
Airline_Tweets = './airline.csv'
df = pd.read_csv(Airline_Tweets)
df.head(10)

,weekday,month,date,year,text
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA"
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA"
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3"
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3"
5,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3
6,Thu,Oct,1,2015,@BigGucciQueen This is your direct dial number + 43 (0)1 360 277 3461. *DD
7,Thu,Oct,1,2015,@marxypoo ...for any inconvenience. *JH 3/3
8,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3
9,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377 entries, 0 to 1376
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   weekday  1377 non-null   object
 1   month    1377 non-null   object
 2   date     1377 non-null   int64 
 3   year     1377 non-null   int64 
 4   text     1377 non-null   object
dtypes: int64(2), object(3)
memory usage: 53.9+ KB


1. What is the average length of a social customer service reply ?

In [ ]:
df['text'].str.len().mean()
print("Average length of a social customer service reply is", df['text'].str.len().mean(),".")

Average length of a social customer service reply is 92.1445170660857 .


2. What type of links were referenced most often ?

In [ ]:
counts = {"url": 0, "phone": 0, "dm": 0}

counts['url_links'] = df['text'].str.count('http').sum()
counts['Phone_numbers'] = df['text'].str.count(r"\d{3}-\d{3}-\d{4}").sum()
counts['Direct_messages'] = df['text'].str.count('DM').sum()

max_type = max(counts, key=counts.get)

print("The type referenced most often:", max_type)

The type referenced most often: Direct_messages


3. How many people should be on a social media customer service team? 

In [ ]:
# Find all the name of customer service representative in reply * Two capital letters at the end of each reply
name = df['text'].str.extract(r'\*(\w+)', expand=False).unique()
print(name)
# Count all unique name of customer service representative in reply
count = df['text'].str.extract(r'\*(\w+)', expand=False).nunique()

print("There should be around",(count // 10) * 10,"people on a social media customer service team.")


['AA' 'JH' nan 'DD' 'RD' 'WG' 'NG' 'AB' 'MR' 'BB' 'KM' 'KC' 'TH' 'RB' 'SD'
 'MD' 'VM' 'QB' 'PL' 'RS' 'CS' 'JJ' 'SB' 'EC' 'DR' 'LS' 'ML' 'CK' 'HW'
 'TP' 'VI' 'AN' 'CM' 'AD']
There should be around 30 people on a social media customer service team.


4. How many social replies are reasonable for a customer service representative to handle?

In [ ]:
# Caculate reply per each customer service representative
cs_NAME = df['text'].str.extract(r'\*(\w+)', expand=False).value_counts()
reply_count = pd.DataFrame({'cs_NAME': cs_NAME.index, 'counts': cs_NAME.values})

print("A customer service representative is likely to handle about",reply_count.mean(),"replies per day.")

A customer service representative is likely to handle about counts   35.76
dtype: float64 replies per day.


# **CASE II – Exploring the Imbalanced Dataset Problem**
##Java Development Tools Bug Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
file = "eclipse_jdt.csv"
df= pd.read_csv(file)
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45296 entries, 0 to 45295
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Issue_id          45296 non-null  int64  
 1   Priority          45296 non-null  object 
 2   Component         45296 non-null  object 
 3   Duplicated_issue  7688 non-null   float64
 4   Title             45296 non-null  object 
 5   Description       45253 non-null  object 
 6   Status            45296 non-null  object 
 7   Resolution        45296 non-null  object 
 8   Version           45296 non-null  object 
 9   Created_time      45296 non-null  object 
 10  Resolved_time     45296 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.8+ MB


,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1518,P1,Debug,NaN,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; disabled & hover icons for the following actions:; \t\tCopyToClipboardActionDelegate; \t\tRelaunchActionDelegate; \t\tTerminateAndRemoveActionDelegate; ...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2001-10-18 11:51:14 -0400
1,1519,P3,Debug,NaN,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint in InfiniteLoop with a hit count. When suspended;; \tright click breakpoint in Breakpoints viewer and enable. Hit count; \tstays at previous value. ...,RESOLVED,WORKSFORME,2.0,2001-10-10 22:14:00 -0400,2001-11-28 13:42:46 -0500
2,1520,P3,Debug,NaN,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users to use styled text in console; with preference support. This would; \tallow console input to be shown in bold; etc.,CLOSED,WONTFIX,2.0,2001-10-10 22:14:00 -0400,2002-06-26 11:32:05 -0400


##STEP 1 - Recode the outcome into a binary variable

In [ ]:
# Recode the outcome into a binary variable
df['new_priority'] = df['Priority'].apply(lambda x: 1 if x == 'P3' else 0)

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)


##STEP 2 - Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Replace missing values with empty string in the train data
train_df['Description'] = train_df['Description'].fillna('')

# Replace missing values with empty string in the test data
test_df['Description'] = test_df['Description'].fillna('')

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Transform the train data using the vectorizer
X_train = vectorizer.fit_transform(train_df['Description'])
y_train = train_df['new_priority']

# Transform the test data using the same vectorizer
X_test = vectorizer.transform(test_df['Description'])
y_test = test_df['new_priority']


##STEP 3 - Build Reference Results

In [ ]:
# Keeping the trivial classifier as a baseline, select 5 different classifiers to be compared
classifiers = [    ("Baseline", DummyClassifier(strategy="most_frequent")),    ("MultinomialNB", MultinomialNB()),    ("LogisticRegression", LogisticRegression(max_iter=1000)),    ("DecisionTreeClassifier", DecisionTreeClassifier()),    ("RandomForestClassifier", RandomForestClassifier(n_estimators=100))]

# Evaluate the performance of each classifier on the test set
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
 

    # Print evaluation results
    print(name)
    print("Accuracy:", accuracy)
    print("F1-Score:", f1)
    print("ROC AUC:", roc_auc)
    print("Precision:", precision)
    print("Recall:", recall)
    print("\n")
  

Baseline
Accuracy: 0.8738685701670469
F1-Score: 0.932689286836318
ROC AUC: 0.5
Precision: 0.8738685701670469
Recall: 1.0


MultinomialNB
Accuracy: 0.8738685701670469
F1-Score: 0.932689286836318
ROC AUC: 0.5
Precision: 0.8738685701670469
Recall: 1.0


LogisticRegression
Accuracy: 0.8757082934726617
F1-Score: 0.9332490218551159
ROC AUC: 0.5242663636921943
Precision: 0.8792820971105153
Recall: 0.9942736842105263


DecisionTreeClassifier
Accuracy: 0.8186032820663772
F1-Score: 0.8971674106211672
ROC AUC: 0.5609842657986857
Precision: 0.8889715608465608
Recall: 0.9055157894736842


RandomForestClassifier
Accuracy: 0.8750459930826404
F1-Score: 0.933096926713948
ROC AUC: 0.5131541853466806
Precision: 0.8767863754165124
Recall: 0.9971368421052632




##STEP 4 - Undersampling the majority class

In [ ]:
# Create an instance of RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = rus.fit_resample(X_train, train_df['new_priority'])

# Train a new Naive Bayes model on the resampled data
nb = MultinomialNB()
nb.fit(X_resampled, y_resampled)

# Predict the class labels for the test data
y_pred = nb.predict(X_test)

# Evaluate the model performance on the test data
accuracy = accuracy_score(test_df['new_priority'], y_pred)
f1_score = f1_score(test_df['new_priority'], y_pred)
roc_auc = roc_auc_score(test_df['new_priority'], y_pred)
precision = precision_score(test_df['new_priority'], y_pred)
recall = recall_score(test_df['new_priority'], y_pred)

# Print evaluation results
print("Undersampling Results:")
print("Accuracy:", accuracy)
print("F1-Score:", f1_score)
print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)



Undersampling Results:
Accuracy: 0.6190300978732799
F1-Score: 0.7365796570498143
ROC AUC: 0.6472316403611128
Precision: 0.9305734121882232
Recall: 0.6095157894736842


##STEP 5 - Oversampling the minority class

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import (accuracy_score, f1_score, cohen_kappa_score,
                             roc_auc_score, average_precision_score,
                             precision_recall_curve, precision_recall_fscore_support)

# Initialize the oversampler
oversampler = RandomOverSampler(random_state=42)

# Resample the training data using the oversampler
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, train_df['new_priority'])

# Train the model on the oversampled data
nb.fit(X_train_oversampled, y_train_oversampled)

# Make predictions on the test data
y_pred = nb.predict(X_test)

# Evaluate the model on the test data
accuracy = accuracy_score(test_df['new_priority'], y_pred)
f1 = f1_score(test_df['new_priority'], y_pred)
roc_auc = roc_auc_score(test_df['new_priority'], y_pred)
precision = precision_score(test_df['new_priority'], y_pred)
recall = recall_score(test_df['new_priority'], y_pred)


# Print evaluation results
print("Oversampling Results:")
print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)



Oversampling Results:
Accuracy: 0.6984325557436162
F1-Score: 0.8055054579971523
ROC AUC: 0.6504791254682799
Precision: 0.9228928765633496
Recall: 0.7146105263157895


##STEP 6 - SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

# Create SMOTE object
smote = SMOTE(random_state=42)

# Resample training data using SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Create logistic regression object
clf = LogisticRegression(random_state=42, max_iter=1000)

# Train and predict on the resampled training data
clf.fit(X_train_resampled, y_train_resampled)
y_pred = clf.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(test_df['new_priority'], y_pred)
f1_s = f1_score(test_df['new_priority'], y_pred)
roc_auc = roc_auc_score(test_df['new_priority'], y_pred)
precision = precision_score(test_df['new_priority'], y_pred)
recall = recall_score(test_df['new_priority'], y_pred)


# Print evaluation results
print("SMOTE")
print("Accuracy:", accuracy)
print("F1 Score:", f1_s)
print("ROC AUC Score:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)



SMOTE
Accuracy: 0.7733460887482523
F1 Score: 0.8645915765409302
ROC AUC Score: 0.6112205858871216
Precision: 0.9045166038082973
Recall: 0.8280421052631579


# **Case III: An exploration of the impact of vectorization on classification**

In [ ]:
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

# common imports
import pandas as pd
import numpy as np
import math
import re
import glob
import os
import sys
import json
import random
import pprint as pp
import textwrap
import sqlite3
import logging

import spacy
import nltk

from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()

# pandas display options
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#available-options
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
pd.options.display.max_colwidth = 200 # default 50; -1 = all
# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# np.set_printoptions(edgeitems=3) # default 3

import matplotlib
from matplotlib import pyplot as plt

plot_params = {'figure.figsize': (8, 4), 
               'axes.labelsize': 'large',
               'axes.titlesize': 'large',
               'xtick.labelsize': 'large',
               'ytick.labelsize':'large',
               'figure.dpi': 100}
# adjust matplotlib defaults
matplotlib.rcParams.update(plot_params)

import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df = pd.read_json("reviews_5_balanced.json", lines=True)
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime
0,1,True,"03 12, 2018",A3QY3THQ42WSCQ,B000YFSR5G,Waaaay too BIG,One Star,1520812800
1,1,True,"03 12, 2018",A3QY3THQ42WSCQ,B000YFSR4W,Waaaay too BIG,One Star,1520812800
2,1,True,"02 8, 2017",A21HH0VIBKK80J,B000YFSR5G,"Was terribly disappointed, the pants were way too large in the legs, my husband looked like he was wearing blown up clown pants.","Was terribly disappointed, the pants were way too large in ...",1486512000
3,1,True,"02 8, 2017",A21HH0VIBKK80J,B000YFSR4W,"Was terribly disappointed, the pants were way too large in the legs, my husband looked like he was wearing blown up clown pants.","Was terribly disappointed, the pants were way too large in ...",1486512000
4,1,True,"02 19, 2018",A276HQXYS553QW,B0014F8TIU,Constantly rolls down,One Star,1518998400


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294240 entries, 0 to 294239
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         294240 non-null  int64 
 1   verified        294240 non-null  bool  
 2   reviewTime      294240 non-null  object
 3   reviewerID      294240 non-null  object
 4   asin            294240 non-null  object
 5   reviewText      294240 non-null  object
 6   summary         294132 non-null  object
 7   unixReviewTime  294240 non-null  int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 16.0+ MB


# Big Question: Does the type of vectorization selected have a significant impact on the performance of the classification model?

### Data Cleaning

In [ ]:
# Check for missing values
print(df.isnull().sum())

overall             0
verified            0
reviewTime          0
reviewerID          0
asin                0
reviewText          0
summary           108
unixReviewTime      0
dtype: int64


In [ ]:
# Drop rows with missing values
df = df.dropna()

# Fill missing values with an empty string
df['summary'] = df['summary'].fillna('')

In [ ]:
df.nunique()

overall                4
verified               2
reviewTime          3071
reviewerID        181296
asin              126303
reviewText        222666
summary            84650
unixReviewTime      3071
dtype: int64

The 'overall' column does not have a rating = 3

In [ ]:
df['overall'].value_counts()

5    127110
1     84805
2     59363
4     22854
Name: overall, dtype: int64

In [ ]:
# Create a binary outcome column keeping 1 for positive reviews and 0 for negative reviews
df["binary_output"] = df["overall"].apply(lambda x: 1 if x > 3 else 0)

The target class looks balanced

In [ ]:
df['binary_output'].value_counts(normalize=False)

1    149964
0    144168
Name: binary_output, dtype: int64

### Text Pre-processing on reviewText column

In [ ]:
# Define a function for text cleaning and pre-processing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize the text
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join the tokens back into a string
    text = ' '.join(tokens)
    return text

# Apply the preprocess_text function to the reviewText column
df['reviewText'] = df['reviewText'].apply(preprocess_text)

In [ ]:
df.sample(5)

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime,binary_output
215877,4,True,"01 25, 2017",A3VKYWA71F3LH,B00NCXYWLU,great,Four Stars,1485302400,1
146664,5,True,"06 17, 2016",A2E1P5FSF37JSR,B00JRFAMNO,wife loved simple assemble,Nice Planter Box,1466121600,1
260999,5,True,"08 14, 2015",A291BFLK51WJ2C,B00SL1TI8K,great truck,Five Stars,1439510400,1
106088,1,True,"01 23, 2018",AOYT6CHI4Z6IF,B012S22SD4,made sick,One Star,1516665600,0
56963,1,True,"08 20, 2014",A2LJFS1M8Z3E9H,B00IT7I1MA,thick credit card compartment tight could get one card without getting stuck,Was very thick and the credit card compartment was very ...,1408492800,0


### Split data into training and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['binary_output'], test_size=0.2, random_state=42)

### Define various text vectorization techniques


*   TF-IDF Vectorizer
*   Bag of Vectorization with binary weights
*   Topic Modelling with Non-negative matrix factorization
*   Topic Modelling with Latent Semantic Analysis (LSA)
*   Topic Modelling with Latent Dirichlet Allocation (LDA)



In [ ]:
# 1. Using TF-IDF Vectorizer by defining max document frequency and min document frequency
tfidf_vec = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=10)

# 2. Using Bag of Words Vectorization technique with binary weights
count_vec = CountVectorizer(binary=True)

# 3. Vectorization using Topic Modeling with Non-Negative Matrix Factorization 
nmf_vec = NMF(n_components=10, random_state=42)

# 4. Vectorization using Topic Modeling with Latent Semantic Analysis (LSA)
lsa_vec = TruncatedSVD(n_components=10, algorithm='randomized', random_state=42)

# 5. Vectorization using Topic Modeling with Latent Dirichlet Allocation (LDA)
lda_vec = LatentDirichletAllocation(n_components=10, random_state=42)


### Fit the vectors on train and test sets

In [ ]:
# Apply the vectorizers on the train data
tfidf_train = tfidf_vec.fit_transform(X_train)
count_train = count_vec.fit_transform(X_train)
nmf_train = nmf_vec.fit_transform(tfidf_train)
lsa_train = lsa_vec.fit_transform(tfidf_train)
lda_train = lda_vec.fit_transform(tfidf_train)

# Apply the vectorizers on the test data
tfidf_test = tfidf_vec.transform(X_test)
count_test = count_vec.transform(X_test)
nmf_test = nmf_vec.transform(tfidf_test)
lsa_test = lsa_vec.transform(tfidf_test)
lda_test = lda_vec.transform(tfidf_test)

### Define classification models:
1. Random Forest
2. Logistic Regression
3. XGBoost

In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Set up models to loop through
models = [(RandomForestClassifier(n_jobs=-1, random_state=42), 'Random Forest Classifier'), 
          (LogisticRegression(random_state=42), 'Logistic Regression'), 
          (XGBClassifier(), 'XGB Classifier')]

# models = [(XGBClassifier(), 'XGB Classifier')]

# Set up vectorizers to loop through
vectorizers = [(tfidf_train, tfidf_test, 'TF-IDF'), 
               (count_train, count_test, 'Count'), 
               (nmf_train, nmf_test, 'NMF'), 
               (lsa_train, lsa_test, 'LSA'), 
               (lda_train, lda_test, 'LDA')]

# Loop through models and vectorizers
for model in models:
    for vectorizer in vectorizers:
        print(f'\n Training {model[1]} on {vectorizer[2]} vectorized data')
        # Train model on vectorized data
        model[0].fit(vectorizer[0], y_train)
        # Make predictions and calculate accuracy
        pred = model[0].predict(vectorizer[1])
        acc = accuracy_score(y_test, pred)
        print(pd.crosstab(y_test, pred, rownames=['True'], colnames=['Predicted'], margins=True))
        print(classification_report(y_test, pred))
        print(f"{model[1]} trained on {vectorizer[2]} vectorized data has an accuracy of {acc} \n\n")
        


 Training Random Forest Classifier on TF-IDF vectorized data
Predicted      0      1    All
True                          
0          24950   3970  28920
1           3825  26082  29907
All        28775  30052  58827
              precision    recall  f1-score   support

           0       0.87      0.86      0.86     28920
           1       0.87      0.87      0.87     29907

    accuracy                           0.87     58827
   macro avg       0.87      0.87      0.87     58827
weighted avg       0.87      0.87      0.87     58827

Random Forest Classifier trained on TF-IDF vectorized data has an accuracy of 0.8674928179237426 



 Training Random Forest Classifier on Count vectorized data
Predicted      0      1    All
True                          
0          25489   3431  28920
1           3746  26161  29907
All        29235  29592  58827
              precision    recall  f1-score   support

           0       0.87      0.88      0.88     28920
           1       0.88      0.

### From the above results, the Random Forest Classifier trained on Count vectorized data (i.e, Bag of Words Vectorization technique with binary weights) achieved the highest accuracy of 0.8779


### Thus, the type of vectorization selected has a significant impact on the performance of the classification model